In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA

xgb.set_config(verbosity=0)
assert xgb.get_config()["verbosity"] == 0

In [12]:
train_df = pd.read_csv('train.csv')
train_df.sample(5)

,uid,NAME_CONTRACT_TYPE,TARGET,n_loans,total_loan_amount,mean_loan_amount,min_loan_amount,max_loan_amount,amount_overdue,overdue_ratio,...,late_days,late_months,loan_frequency,days_since_last_loan,n_enquiries,total_enquiry_amount,mean_enquiry_amount,min_enquiry_amount,max_enquiry_amount,enquiry_frequency
13108,MIB98801892,Cash loans,0,20,6776077.50,338803.875000,0.0,2316600.00,0.0,0.0,...,62,2,0.018639,0,3,301000,100333.333333,56000,130000,0.043478
50863,XGP93369788,Revolving loans,0,2,245250.00,122625.000000,0.0,245250.00,0.0,0.0,...,0,0,0.014286,0,3,959139,319713.000000,36446,476252,0.013889
122774,EUN00045137,Cash loans,0,2,93117.51,46558.755000,0.0,93117.51,0.0,0.0,...,0,0,0.002000,0,4,358000,89500.000000,26000,146000,0.012121
43431,QIO00079383,Cash loans,0,7,4671049.50,667292.785714,28332.0,4216500.00,0.0,0.0,...,0,0,0.003291,0,2,154000,77000.000000,18000,136000,0.015504
88556,BOE25885836,Cash loans,0,4,1484100.00,371025.000000,90000.0,989100.00,0.0,0.0,...,141,3,0.002468,0,7,902000,128857.142857,38000,192000,0.010989


In [13]:
train_df['NAME_CONTRACT_TYPE'] = train_df['NAME_CONTRACT_TYPE'].map({'Cash loans': 0, 'Revolving loans': 1})

In [14]:
X = train_df.loc[:, (train_df.columns != 'TARGET') & (train_df.columns != 'uid')]
y = train_df['TARGET']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=94)

## XGBoost

In [18]:
clf = xgb.XGBClassifier(scale_pos_weight=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(roc_auc_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

0.5800149586029922


array([[40233, 11433],
       [ 2669,  1645]], dtype=int64)

## SVC

In [20]:
pca = PCA(n_components=5)
X_train_transformed = pca.fit_transform(X_train)
X_test_transformed = pca.transform(X_test)

In [ ]:
clf = SVC(gamma='scale', class_weight='balanced')
clf.fit(X_train_transformed[:10000], y_train[:10000])
y_pred = clf.predict(X_test_transformed)
print(roc_auc_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

In [12]:
y_pred = clf.predict(X_test_transformed)
confusion_matrix(y_test, y_pred)

array([[28662, 23004],
       [ 2266,  2048]], dtype=int64)